In [ ]:
import os
from dotenv import load_dotenv
import assemblyai as aai
from pydantic import BaseModel
from openai import OpenAI
import json
from datetime import datetime

from prompt import system_prompt
load_dotenv()

In [ ]:
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
ASSEMBLY_KEY = os.getenv("ASSEMBLY_KEY")

In [ ]:
def transcribe_audio(audio_url):
    aai.settings.api_key = ASSEMBLY_KEY

    config = aai.TranscriptionConfig(speech_model=aai.SpeechModel.universal, speaker_labels=True, language_code="es", speakers_expected=2)

    transcript = aai.Transcriber(config=config).transcribe(audio_file)
    if transcript.status == "error":
        raise RuntimeError(f"Transcription failed: {transcript.error}")

    full_text = ""
    for utterance in transcript.utterances:
        speaker = f"Speaker{utterance.speaker}"
        text = utterance.text
        
        full_text += f"{speaker}: {text}\n\n"
    
    return full_text
    

In [ ]:
def generate_temporal_context():
    hoy = datetime.now()
    dia_semana = hoy.strftime("%A")
    fecha = hoy.strftime("%d de %B de %Y %H:%M")
    return f"hoy es {dia_semana}, {fecha}."

In [ ]:
def generate_historia_clinica(transcription, prompt):
    client = OpenAI(api_key=OPENAI_API_KEY)

    temporal_context = generate_temporal_context()
    formatted_prompt = prompt.format(temporal_context=temporal_context)

    completion = client.responses.create(
        model="gpt-5",
        reasoning={"effort": "minimal"},
        input=[
            {
                "role": "system",
                "content": formatted_prompt,
            },
            {
                "role": "user",
                "content": (
                    transcription
                ),
            },
        ],
        text={"format": {"type": "json_object"}},
    )
    
    data = json.loads(completion.output[1].content[0].text)
    return data

In [ ]:
audio_path=""

In [ ]:
transcription = transcribe_audio(audio_path)
histoiria_clinica = generate_historia_clinica(transcription, system_prompt)
histoiria_clinica